<a href="https://colab.research.google.com/github/sujeengim/dogs-vs-cats/blob/main/mini_cvd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# dogs vs cats mini dataset
c - cat, 0

d - dog, 1

# 1. 데이터 준비

In [93]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [94]:
from sklearn.model_selection import train_test_split

import os
import pandas as pd

In [122]:
c_filenames = os.listdir('/content/drive/MyDrive/mapinskhu/AI-test.ver/mini-data/cats_set')
c_df = pd.DataFrame({
    'filename': c_filenames,
    'category':0,
})

In [123]:
d_filenames = os.listdir('/content/drive/MyDrive/mapinskhu/AI-test.ver/mini-data/dogs_set')
d_df = pd.DataFrame({
    'filename': d_filenames,
    'category':1,
})

In [97]:
# d_df

In [124]:
cnd_df = pd.concat([c_df, d_df], ignore_index=True)
cnd_df_file = cnd_df['filename']
cnd_df_target = cnd_df['category']

In [99]:
cnd_df_file

,filename
0,cat.4010.jpg
1,cat.4001.jpg
2,cat.4008.jpg
3,cat.4007.jpg
4,cat.4004.jpg
...,...
995,dog.4481.jpg
996,dog.4497.jpg
997,dog.4498.jpg
998,dog.4499.jpg


In [125]:
train_file, val_file, train_target, val_target,  = train_test_split(
    cnd_df_file, cnd_df_target, test_size=0.3, random_state=42, stratify=cnd_df_target
)

In [101]:
# train_file

In [102]:
# train_target

In [126]:
train_df = pd.concat([train_file, train_target], axis=1).reset_index(drop=True)
val_df = pd.concat([val_file, val_target], axis=1).reset_index(drop=True)

In [104]:
# train_df

In [105]:
# 카테고리 비율 확인
# train_df['category'].value_counts().plot.bar()

In [127]:
train_df['category'] = train_df['category'].replace({0:'cat', 1:'dog'})
val_df['category'] = val_df['category'].replace({0:'cat', 1:'dog'})

In [128]:
train_df

,filename,category
0,dog.4156.jpg,dog
1,cat.4008.jpg,cat
2,dog.4203.jpg,dog
3,cat.4306.jpg,cat
4,dog.4228.jpg,dog
...,...,...
695,cat.4342.jpg,cat
696,dog.4399.jpg,dog
697,cat.4139.jpg,cat
698,cat.4250.jpg,cat


In [136]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  700 non-null    object
 1   category  700 non-null    object
dtypes: object(2)
memory usage: 11.1+ KB


# 1-2. train, val 데이터 저장

In [ ]:
from PIL import Image
for name in train_df['filename']:
    if name.split('.')[0] == 'cat':
      img = Image.open('/content/drive/MyDrive/mapinskhu/AI-test.ver/mini-data/cats_set/'+name)
    else:
      img = Image.open('/content/drive/MyDrive/mapinskhu/AI-test.ver/mini-data/dogs_set/'+name)
    img.save('/content/drive/MyDrive/mapinskhu/AI-test.ver/mini-data/train/'+name, 'jpeg')

for name in val_df['filename']:
    if name.split('.')[0] == 'cat':
      img = Image.open('/content/drive/MyDrive/mapinskhu/AI-test.ver/mini-data/cats_set/'+name)
    else:
      img = Image.open('/content/drive/MyDrive/mapinskhu/AI-test.ver/mini-data/dogs_set/'+name)
    img.save('/content/drive/MyDrive/mapinskhu/AI-test.ver/mini-data/val/'+name, 'jpeg')

# 1-3. 데이터 제너레이터

In [108]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [109]:
img_width = 128
img_height = 128
img_channels = 3

In [110]:
train_gen = ImageDataGenerator(rescale=1./255)
val_gen = ImageDataGenerator(rescale=1./255)

In [120]:
train_generator = train_gen.flow_from_dataframe(
    train_df,
    x_col='filename', y_col='category',
    target_size=(img_width, img_height),
    class_mode='categorical',
    batch_size=100
)

Found 0 validated image filenames belonging to 0 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 700 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


# 2. cnn 모델 만들기

In [112]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, MaxPooling2D, Dropout, BatchNormalization

In [113]:
model = Sequential()

model.add(Conv2D(32, (3,3), activation='relu', input_shape=(img_width, img_height, img_channels)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_7 (Conv2D)                    │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 126, 126, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_9                │ (None, 61, 61, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_10               │ (None, 28, 28, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 512)                 │      12,845,568 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_11               │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 2)                   │           1,026 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,942,786 (49.37 MB)

 Trainable params: 12,941,314 (49.37 MB)

 Non-trainable params: 1,472 (5.75 KB)

In [114]:
# 콜백(조기종료, 학습율 하향조정)
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [115]:
earlystop = EarlyStopping(patience=10)

In [116]:
learning_rate_reduction = ReduceLROnPlateau(monitor=';val_acc',
                                            patience=2,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.00001)

In [117]:
callbacks = [earlystop, learning_rate_reduction]

# 3. 학습 시작

In [118]:
epoch = 20

In [121]:
history =model.fit(
    train_generator,
    epochs=epoch,
    validation_data=val_df,
    callbacks=callbacks,
    #validation_steps=total_validate//BATCH_SIZE,
    #steps_per_epoch=total_train/BATCH_SIZE,

)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


ValueError: The PyDataset has length 0